# When to Use SFT
Before diving into implementation, it’s important to understand when SFT is the right choice for your project. As a first step, you should consider whether using an existing instruction-tuned model with well-crafted prompts would suffice for your use case. SFT involves significant computational resources and engineering effort, so it should only be pursued when prompting existing models proves insufficient.
> Consider SFT only if you: - Need additional performance beyond what prompting can achieve - Have a specific use case where the cost of using a large general-purpose model outweighs the cost of fine-tuning a smaller model - Require specialized output formats or domain-specific knowledge that existing models struggle with

If you determine that SFT is necessary, the decision to proceed depends on two primary factors:

## Template Control
SFT allows precise control over the model’s output structure. This is particularly valuable when you need the model to:
1. Generate responses in a specific chat template format
2. Follow strict output schemas
3. Maintain consistent styling across responses

## Domain Adaptation
When working in specialized domains, SFT helps align the model with domain-specific requirements by:
1. Teaching domain terminology and concepts
2. Enforcing professional standards
3. Handling technical queries appropriately
4. Following industry-specific guidelines
> Before starting SFT, evaluate whether your use case requires: - Precise output formatting - Domain-specific knowledge - Consistent response patterns - Adherence to specific guidelines
This evaluation will help determine if SFT is the right approach for your needs.

In [ ]:
# Install the requirements in Google Colab
!pip -q install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

# Implementation with TRL
Now that we understand the key components, let’s implement the training with proper validation and monitoring. We will use the SFTTrainer class from the Transformers Reinforcement Learning (TRL) library, which is built on top of the transformers library. Here’s a complete example using the TRL library:

In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
import torch

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load dataset
dataset = load_dataset("HuggingFaceTB/smoltalk", "all")

# Configure model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(
    device
)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
# Setup chat template
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Configure trainer
training_args = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
)

# Start training
trainer.train()



> When using a dataset with a "messages" field (like the example above), the SFTTrainer automatically applies the model's chat template, which it retrieves from the hub. This means you don't need any additional configuration to handle chat-style conversations - the trainer will format the messages according to the model's expected template format.



# Packing the Dataset
The SFTTrainer supports example packing to optimize training efficiency. This feature allows multiple short examples to be packed into the same input sequence, maximizing GPU utilization during training. To enable packing, simply set packing=True in the SFTConfig constructor. When using packed datasets with max_steps, be aware that you may train for more epochs than expected depending on your packing configuration. You can customize how examples are combined using a formatting function - particularly useful when working with datasets that have multiple fields like question-answer pairs. For evaluation datasets, you can disable packing by setting eval_packing=False in the SFTConfig. Here’s a basic example of customizing the packing configuration:

In [ ]:
# Configure packing
training_args = SFTConfig(packing=True)

trainer = SFTTrainer(model=model, train_dataset=dataset, args=training_args)

trainer.train()

When packing the dataset with multiple fields, you can define a custom formatting function to combine the fields into a single input sequence. This function should take a list of examples and return a dictionary with the packed input sequence. Here’s an example of a custom formatting function:

In [ ]:
def formatting_func(example):
    text = f"### Question: {example['question']}\n ### Answer: {example['answer']}"
    return text


training_args = SFTConfig(packing=True)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=training_args,
    formatting_func=formatting_func,
)

# Metrics to Monitor
Effective monitoring involves tracking quantitative metrics, and evaluating qualitative metrics. Available metrics are:

* Training loss
* Validation loss
* Learning rate progression
* Gradient norms


> Watch for these warning signs during training: 1. Validation loss increasing while training loss decreases (overfitting) 2. No significant improvement in loss values (underfitting) 3. Extremely low loss values (potential memorization) 4. Inconsistent output formatting (template learning issues)

# Warning Signs to Watch For
Several patterns in the loss curves can indicate potential issues. Below we illustrate common warning signs and solutions that we can consider.



> If the validation loss decreases at a significantly slower rate than training loss, your model is likely overfitting to the training data. Consider:
1. Reducing the training steps
2. Increasing the dataset size
3. Validating dataset quality and diversity



> If the loss doesn’t show significant improvement, the model might be:
1. Learning too slowly (try increasing the learning rate)
2. Struggling with the task (check data quality and task complexity)
3. Hitting architecture limitations (consider a different model)



> Extremely low loss values could suggest memorization rather than learning. This is particularly concerning if:
1. The model performs poorly on new, similar examples
2. The outputs lack diversity
3. The responses are too similar to training examples

Monitor both the loss values and the model's actual outputs during training. Sometimes the loss can look good while the model develops unwanted behaviors. Regular qualitative evaluation of the model's responses helps catch issues that metrics alone might miss.











# Evaluation after SFT

After completing SFT, consider these follow-up actions:

1. Evaluate the model thoroughly on held-out test data
2. Validate template adherence across various inputs
3. Test domain-specific knowledge retention
4. Monitor real-world performance metrics
> Document your training process, including: - Dataset characteristics - Training parameters - Performance metrics - Known limitations This documentation will be valuable for future model iterations.

# TRL - Transformer Reinforcement Learning
TRL is a full stack library where we provide a set of tools to train transformer language models with methods like Supervised Fine-Tuning (SFT), Group Relative Policy Optimization (GRPO), Direct Preference Optimization (DPO), Reward Modeling, and more. The library is integrated with 🤗 transformers.

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [ ]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [ ]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    # tokenizer=tokenizer,
    processing_class=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# TODO: use the fine-tuned to model generate a response, just like with the base example.
# Load the fine-tuned model and tokenizer
model_name = "Mhammad2023/SmolLM2-FT-MyDataset"
model_fine = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer_fine = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
# model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)
outputs = model_fine.generate(**inputs, max_new_tokens=100)
print("After training:")
print(tokenizer_fine.decode(outputs[0], skip_special_tokens=True))

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.

# 🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

# Set device
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

# Load model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set up chat format (important for TRL-style fine-tuning)
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)



In [ ]:
# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-BigCode-Python"
finetune_tags = ["smol-course", "module_2"]

In [ ]:
# Load the dataset (Python subset only)
ds = load_dataset("bigcode/the-stack-smol", data_dir="data/python")
ds

In [ ]:
ds = ds["train"]
ds

In [ ]:
def convert_to_chat(example):
    parts = example["repository_name"].split("/")
    repo_name = parts[1] if len(parts) == 2 else parts[0]

    return {
        "messages": [
            {
                "role": "user",
                "content": f"Generate code for `{repo_name}/{example['path']}`."
            },
            {"role": "assistant", "content": example["content"]}
        ]
    }

ds = ds.map(convert_to_chat)
ds

In [ ]:
ds[5]['repository_name']

In [ ]:
ds[5]['path']

In [ ]:
ds[5]['messages']

In [ ]:
# 🛠️ Preprocess dataset: use only the `content` column
# now it's messages
def preprocess(example):
    return {"messages": example["messages"]}

ds = ds.map(preprocess, remove_columns=ds.column_names)
ds

In [ ]:
ds[0]['messages']

In [ ]:
# Shuffle and split into train/test manually
split = ds.train_test_split(test_size=0.2, seed=42)

ds = {
    "train": split["train"],
    "test": split["test"]
}
ds

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    # max_steps=1000,  # Adjust based on dataset size and desired training duration
    # per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    # learning_rate=5e-5,  # Common starting point for fine-tuning
    max_steps=500,
    learning_rate=2e-5,
    warmup_steps=50,
    max_length=512,
    per_device_train_batch_size=2,     # ✅ LOWERED for GPU RAM
    gradient_accumulation_steps=4,     # ✅ Accumulate gradients for effective batch size of 8
    save_total_limit=2,               # ✅ Prevent disk from filling up
    fp16=True,                        # ✅ Enable mixed precision if on CUDA
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)


**Use gradient_accumulation_steps**
Simulates larger batch size:

**batch_size** = per_device_train_batch_size × gradient_accumulation_steps

Example: 2 × 4 = 8 effective batch size

In [ ]:
print(tokenizer.chat_template)


In [ ]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
)

In [ ]:
# Train
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

In [ ]:
# Load the fine-tuned model and tokenizer
model_name = "Mhammad2023/SmolLM2-FT-BigCode-Python"
model_fine = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer_fine = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)


In [ ]:
prompt = "Generate code for `machinelearning/decisiontree.py`."
messages = [{"role": "user", "content": prompt}]
input_text = tokenizer_fine.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer_fine(input_text, return_tensors="pt").input_ids.to(model_fine.device)

outputs = model_fine.generate(
    input_ids=input_ids,
    max_new_tokens=2048,
    temperature=0.7,
    do_sample=True,
)

print(tokenizer_fine.decode(outputs[0], skip_special_tokens=True))


In [ ]:
prompt = (
    "Write a Python script to train a Decision Tree Classifier using scikit-learn."
)
messages = [{"role": "user", "content": prompt}]
input_text = tokenizer_fine.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer_fine(input_text, return_tensors="pt").input_ids.to(model_fine.device)

eos_token_id = tokenizer_fine.eos_token_id
outputs = model_fine.generate(
    input_ids=input_ids,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    eos_token_id=eos_token_id,
)

output_text = tokenizer_fine.decode(outputs[0], skip_special_tokens=True).strip()
print(output_text)